In [ ]:
# 03-Model-Inference.ipynb

import mlflow
from pyspark.sql import SparkSession
from pyspark.sql.functions import struct

# मॉडल का नाम और स्टेज परिभाषित करना
model_name = "HousePricePredictionModel"
# यह स्टेज बदलती रहेगी (Staging/Production)
model_stage = "Production" 

# मॉडल रजिस्ट्री से मॉडल लोड करना
# यह UAT/Prod वातावरण का अनुकरण करता है
model = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_stage}")

print(f"{model_stage} स्टेज से मॉडल सफलतापूर्वक लोड हो गया है।")

# डेल्टा टेबल से डेटा लोड करना
delta_table_path = "/tmp/house_price_prediction_data"
spark_df = spark.read.format("delta").load(delta_table_path)

# मॉडल का उपयोग करके प्रेडिक्शन करना
predictions_df = spark_df.toPandas()
predictions_df['predicted_price'] = model.predict(predictions_df[['sq_feet', 'num_bedrooms', 'num_bathrooms', 'year_built', 'location_score']])

print("प्रेडिक्शन सफलतापूर्वक पूरा हो गया है।")

# प्रेडिक्शन के साथ डेटा दिखाना
display(spark.createDataFrame(predictions_df))